In [ ]:
!pip install transformers datasets torch

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Final_Golden_Data.xlsx'

Mounted at /content/drive


In [ ]:
df = pd.read_excel(file_path)
df = df[['text', 'summary']]

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# model_name = "facebook/mbart-large-50-many-to-many-mmt"
# tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
# model = MBartForConditionalGeneration.from_pretrained(model_name)

# tokenizer.src_lang = "ur_PK"
# tokenizer.tgt_lang = "ur_PK"

load_path = "/content/drive/MyDrive/trained_on_Second25k"

# Load the saved model and tokenizer
tokenizer = MBart50TokenizerFast.from_pretrained(load_path)
model = MBartForConditionalGeneration.from_pretrained(load_path)


# Set source and target languages
tokenizer.src_lang = "ur_PK"
tokenizer.tgt_lang = "ur_PK"

print("Model and tokenizer loaded successfully from Google Drive.")

Model and tokenizer loaded successfully from Google Drive.


In [ ]:
# # max_length = 1024

# # def preprocess_data(examples):
# #     # Ensure values are strings and replace NaN with an empty string for both text and summary
# #     inputs = [str(text) if pd.notna(text) else "" for text in examples['text']]
# #     targets = [str(summary) if pd.notna(summary) else "" for summary in examples['summary']]

# #     # Tokenize inputs
# #     model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")

# #     # Tokenize targets with the tokenizer as a target tokenizer
# #     with tokenizer.as_target_tokenizer():
# #         labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")

# #     # Add labels to model inputs
# #     model_inputs["labels"] = labels["input_ids"]
# #     return model_inputs

# # # Tokenize datasets
# # train_dataset = train_dataset.map(preprocess_data, batched=True)
# # test_dataset = test_dataset.map(preprocess_data, batched=True)


# max_length = 1024

# def preprocess_data(examples):
#     inputs = examples['text']
#     targets = examples['summary']
#     model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# # Tokenize datasets
# train_dataset = train_dataset.map(preprocess_data, batched=True)
# test_dataset = test_dataset.map(preprocess_data, batched=True)


max_length = 1024

def preprocess_data(examples):
    # Ensure all inputs and targets are strings and replace NaN with an empty string
    inputs = [str(text) if pd.notna(text) else "" for text in examples["text"]]
    targets = [str(summary) if pd.notna(summary) else "" for summary in examples["summary"]]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")

    # Tokenize targets using `text_target`
    labels = tokenizer(text_target=targets, max_length=max_length, truncation=True, padding="max_length")

    # Add labels to model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/787 [00:00<?, ? examples/s]

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [ ]:
# from transformers import DataCollatorForSeq2Seq
# from torch.utils.data import DataLoader

# # batch_size = 2
# # accumulation_steps = 8

# # ---------------------------------------------------------------------------
# # this is the new code for testing on a larger dataset
# # Set batch size and max length for memory efficiency
# batch_size = 4  # Try lowering to 2 if still out of memory
# max_length = 1024

# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     gradient_accumulation_steps=4,  # Keep same effective batch size
#     num_train_epochs=2,
#     weight_decay=0.01,
#     save_total_limit=2,  # Limit the number of checkpoints
#     save_strategy="steps",  # Save checkpoints every few steps (or use "epoch" to save per epoch)
#     save_steps=500,  # Adjust to save checkpoint every 500 steps
#     push_to_hub=False,
#     fp16=True,
# )

# # Enable gradient checkpointing
# model.gradient_checkpointing_enable()

# # Run this just before training to free up memory
# torch.cuda.empty_cache()

# #  ----------------------------------------------------------------------------

# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest")

# train_dataloader = DataLoader(
#     train_dataset, shuffle=True, collate_fn=data_collator, batch_size=batch_size
# )
# eval_dataloader = DataLoader(
#     test_dataset, collate_fn=data_collator, batch_size=batch_size
# )


from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
import os

output_dir = "/content/drive/MyDrive/results"
os.makedirs(output_dir, exist_ok=True)

# Set batch size and max length for memory efficiency
batch_size = 4  # Try lowering to 2 if still out of memory
max_length = 1024

# Configure TrainingArguments
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,  # Keep same effective batch size
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,  # Limit the number of checkpoints
    save_strategy="steps",  # Save checkpoints every few steps (or use "epoch" to save per epoch)
    save_steps=2000,  # Adjust to save checkpoint every 500 steps
    push_to_hub=False,
    fp16=True,
)

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

# Run this just before training to free up memory
torch.cuda.empty_cache()

# ----------------------------------------------------------------------------
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest")
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=batch_size)
eval_dataloader = DataLoader(test_dataset, collate_fn=data_collator, batch_size=batch_size)

# # Find the latest checkpoint in the output directory
# output_dir = "/content/drive/MyDrive/results"
# checkpoints = [f for f in os.listdir(output_dir) if f.startswith("checkpoint")]
# checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))

# # Check if any checkpoints exist
# if checkpoints:
#     last_checkpoint = os.path.join(output_dir, checkpoints[-1])
#     print(f"Found checkpoint: {last_checkpoint}")
# else:
#     last_checkpoint = None
#     print("No checkpoint found. Training from scratch.")

# import os
# print("Current contents of './results':", os.listdir(output_dir))


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     data_collator=data_collator,
# )

# # Step 5: Train the Model with Optimized Settings
# trainer.train()

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)
trainer.train()
# # Train the model, resuming from last checkpoint if available
# if last_checkpoint:
#     trainer.train(resume_from_checkpoint=last_checkpoint)
# else:
#     trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
0,No log,0.656588
1,No log,0.646117
2,No log,0.647871


TrainOutput(global_step=147, training_loss=0.5611212594168526, metrics={'train_runtime': 206.0667, 'train_samples_per_second': 11.457, 'train_steps_per_second': 0.713, 'total_flos': 5092754954649600.0, 'train_loss': 0.5611212594168526, 'epoch': 2.984771573604061})

In [ ]:
# # Save the model and tokenizer
# model.save_pretrained("/content/drive/My Drive/trained_on_5k")
# tokenizer.save_pretrained("/content/drive/My Drive/trained_on_5k")

# from google.colab import drive
import os

# Mount Google Drive (only needed if not mounted previously)
# drive.mount('/content/drive')

# Define save path and create directory if it doesn’t exist
save_path = "/content/drive/MyDrive/FinalTunedModel"
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer to the specified path
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model and tokenizer saved to {save_path}")


Model and tokenizer saved to /content/drive/MyDrive/FinalTunedModel


In [ ]:
# # Step 8: Evaluation - Generate Summaries for Test Data

# def generate_summary(text):
#     inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to("cuda")
#     summary_ids = model.generate(
#         inputs["input_ids"],
#         max_length=512,  # Shorter summaries for coherence
#         num_beams=5,  # More beams for diversity
#         no_repeat_ngram_size=3,  # Avoid 3-gram repetition
#         length_penalty=1.2,  # Penalize overly long outputs
#         early_stopping=True
#     )
#     return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# # ----------------------------------------------------------------------------------
# # Move model to GPU
# model.to("cuda")

# # Randomly sample 1000 rows from the test set
# sampled_test_df = test_df.sample(n=500, random_state=42)

# # List to store results
# results = []

# # Evaluate on the random samples from the test set
# for _, row in sampled_test_df.iterrows():
#     text = row['text']
#     reference_summary = row['summary']
#     generated_summary = generate_summary(text)

#     # Append the results as a dictionary
#     results.append({
#         "Original Text": text,
#         "Reference Summary": reference_summary,
#         "Generated Summary": generated_summary
#     })

# # Convert results to a DataFrame
# results_df = pd.DataFrame(results)

# # Save results to Excel
# results_df.to_excel("/content/Final_testing_summaries.xlsx", index=False)

# print("Summaries saved to /content/Final_testing_summaries.xlsx")
# Step 8: Evaluation - Generate Summaries for Test Data

def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to("cuda")
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=512,  # Shorter summaries for coherence
        num_beams=5,  # More beams for diversity
        no_repeat_ngram_size=3,  # Avoid 3-gram repetition
        length_penalty=1.2,  # Penalize overly long outputs
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# ----------------------------------------------------------------------------------
# Move model to GPU
model.to("cuda")

# List to store results
results = []

# Evaluate on the entire test set
for _, row in test_df.iterrows():
    text = row['text']
    reference_summary = row['summary']
    generated_summary = generate_summary(text)

    # Append the results as a dictionary
    results.append({
        "Original Text": text,
        "Reference Summary": reference_summary,
        "Generated Summary": generated_summary
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to Excel
output_path = "/content/Final_testing_summaries.xlsx"
results_df.to_excel(output_path, index=False)

print(f"Summaries saved to {output_path}")


Summaries saved to /content/Final_testing_summaries.xlsx


In [ ]:
# from google.colab import files
# files.download("/content/summaries_20k_first_part.xlsx")
from google.colab import files
files.download("/content/Final_testing_summaries.xlsx")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>